In [30]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/brasilian-houses-to-rent/houses_to_rent.csv
/kaggle/input/brasilian-houses-to-rent/houses_to_rent_v2.csv


In [31]:
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport

In [32]:
data= pd.read_csv("../input/brasilian-houses-to-rent/houses_to_rent.csv")
print(data.shape)
data.head(n=5)

(6080, 14)


,Unnamed: 0,city,area,rooms,bathroom,parking spaces,floor,animal,furniture,hoa,rent amount,property tax,fire insurance,total
0,0,1,240,3,3,4,-,acept,furnished,R$0,"R$8,000","R$1,000",R$121,"R$9,121"
1,1,0,64,2,1,1,10,acept,not furnished,R$540,R$820,R$122,R$11,"R$1,493"
2,2,1,443,5,5,4,3,acept,furnished,"R$4,172","R$7,000","R$1,417",R$89,"R$12,680"
3,3,1,73,2,2,1,12,acept,not furnished,R$700,"R$1,250",R$150,R$16,"R$2,116"
4,4,1,19,1,1,0,-,not acept,not furnished,R$0,"R$1,200",R$41,R$16,"R$1,257"


In [33]:
data.drop(data.columns[0], axis=1, inplace=True)

#### Replace '-' in the floor columns

In [34]:
data['floor']=data['floor'].replace(['-'],0)

#### Replace not acept to 0 and acept to 1 in the animal column

In [35]:
data['animal'].replace(to_replace='not acept', value= 0, inplace= True)
data['animal'].replace(to_replace='acept', value= 1, inplace= True)


#### Replace not furnitured to 0 and furnitured to 1 in the furniture column

In [36]:
data['furniture'].replace(to_replace='not furnished', value=0, inplace =True)
data['furniture'].replace(to_replace='furnished', value=1, inplace =True)

#### Clean the columns ('hoa' ,'rent amount' ,'property tax' ,'fire insurance','total')

In [37]:
for col in ['hoa' ,'rent amount' ,'property tax' ,'fire insurance','total']:
    data[col].replace(to_replace='R\$', value='', regex =True, inplace= True)
    data[col].replace(to_replace=',', value='', regex =True, inplace= True)

In [38]:
data['hoa'].replace(to_replace='Sem info', value =0, inplace = True)

In [39]:
data['hoa'].replace(to_replace='Incluso', value =0, inplace = True)

In [40]:
data['property tax'].replace(to_replace='Incluso', value =0, inplace = True)

# Change the data types of columns in the data

In [41]:
data= data.astype(dtype=np.int64)

In [42]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6080 entries, 0 to 6079
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   city            6080 non-null   int64
 1   area            6080 non-null   int64
 2   rooms           6080 non-null   int64
 3   bathroom        6080 non-null   int64
 4   parking spaces  6080 non-null   int64
 5   floor           6080 non-null   int64
 6   animal          6080 non-null   int64
 7   furniture       6080 non-null   int64
 8   hoa             6080 non-null   int64
 9   rent amount     6080 non-null   int64
 10  property tax    6080 non-null   int64
 11  fire insurance  6080 non-null   int64
 12  total           6080 non-null   int64
dtypes: int64(13)
memory usage: 617.6 KB


### Shuffles the Data

In [43]:
data= data.sample(frac=1).reset_index(drop=True)

In [44]:
Y= data.iloc[:, 0]

In [45]:
X=data.iloc[:, 1: ]

In [46]:
X

,area,rooms,bathroom,parking spaces,floor,animal,furniture,hoa,rent amount,property tax,fire insurance,total
0,180,3,2,2,3,1,0,2200,2500,542,32,5274
1,129,4,2,3,2,1,1,1100,5500,520,70,7190
2,38,1,1,0,1,0,0,60,950,0,13,1023
3,130,3,3,2,7,1,0,1200,3500,342,45,5087
4,75,2,2,2,6,1,0,700,1198,227,16,2141
...,...,...,...,...,...,...,...,...,...,...,...,...
6075,63,2,2,1,0,1,0,0,1920,79,29,2028
6076,54,2,1,1,2,0,0,350,1869,0,7,2226
6077,64,1,2,1,3,1,0,763,2603,101,16,3483
6078,200,3,3,5,0,1,0,0,14000,834,211,15050


In [47]:
from sklearn.preprocessing import MinMaxScaler
#MinMax Scaler shrinks the data within the given range, usually of 0 to 1. It transforms data by scaling features to a given range
scaler= MinMaxScaler()
scaler.fit(X)
X=scaler.transform(X)

In [48]:
pd.DataFrame(X)

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.006912,0.222222,0.111111,0.166667,0.030303,1.0,0.0,0.010000,0.046658,0.001480,0.043027,0.012402
1,0.004838,0.333333,0.111111,0.250000,0.020202,1.0,1.0,0.005000,0.113952,0.001420,0.099407,0.017552
2,0.001138,0.000000,0.000000,0.000000,0.010101,0.0,0.0,0.000273,0.011889,0.000000,0.014837,0.000976
3,0.004879,0.222222,0.222222,0.166667,0.070707,1.0,0.0,0.005455,0.069089,0.000934,0.062315,0.011899
4,0.002643,0.111111,0.111111,0.166667,0.060606,1.0,0.0,0.003182,0.017452,0.000620,0.019288,0.003981
...,...,...,...,...,...,...,...,...,...,...,...,...
6075,0.002155,0.111111,0.111111,0.083333,0.000000,1.0,0.0,0.000000,0.033647,0.000216,0.038576,0.003677
6076,0.001789,0.111111,0.000000,0.083333,0.020202,0.0,0.0,0.001591,0.032503,0.000000,0.005935,0.004209
6077,0.002195,0.000000,0.111111,0.083333,0.030303,1.0,0.0,0.003468,0.048968,0.000276,0.019288,0.007588
6078,0.007725,0.222222,0.222222,0.416667,0.000000,1.0,0.0,0.000000,0.304621,0.002277,0.308605,0.038679


In [49]:
from sklearn.model_selection import train_test_split


In [50]:
X_train, X_test, Y_train, Y_test= train_test_split(X, Y, test_size=0.2)

In [51]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

In [52]:
log_model= LogisticRegression( verbose=1)
svc_model= SVC(kernel = 'rbf' ,verbose=1)
nn_model= MLPClassifier(hidden_layer_sizes=(16,16),activation= 'relu',solver= 'adam', verbose=1)

In [53]:
log_model.fit(X_train, Y_train)
svc_model.fit(X_train, Y_train)
nn_model.fit(X_train, Y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
 This problem is unconstrained.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           13     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  3.37147D+03    |proj g|=  1.77800D+03

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
   13     46     53      1     0     0   9.844D-03   1.765D+03
  F =   1765.1030347597350     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             
[LibSVM].
*...
*.
*
optimization finished, #iter = 5094
obj = -1307.886210, rho = 0.993816
nSV = 1507, nBSV = 1194
Total nSV = 1507
Iteration 1, loss = 0.69107823
Iteration 2, loss = 0.54197852


/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


MLPClassifier(hidden_layer_sizes=(16, 16), verbose=1)

In [54]:
print(log_model.score(X_test, Y_test))
print(svc_model.score(X_test, Y_test))
print(nn_model.score(X_test, Y_test))

0.8544407894736842
0.8544407894736842
0.8725328947368421


In [57]:
from sklearn.metrics import f1_score

In [58]:
log_pred= log_model.predict(X_test)
svc_pred= svc_model.predict(X_test)
nn_pred= nn_model.predict(X_test)

In [63]:
print(f1_score(log_pred, Y_test))
print(f1_score(svc_pred, Y_test))
print(f1_score(nn_pred, Y_test))

0.9215077605321508
0.9215077605321508
0.92925604746691


# just checking

In [55]:
data.isin(['Incluso']).any()

city              False
area              False
rooms             False
bathroom          False
parking spaces    False
floor             False
animal            False
furniture         False
hoa               False
rent amount       False
property tax      False
fire insurance    False
total             False
dtype: bool

In [56]:
print(data.shape)
data.head(n=5)

(6080, 13)


,city,area,rooms,bathroom,parking spaces,floor,animal,furniture,hoa,rent amount,property tax,fire insurance,total
0,1,180,3,2,2,3,1,0,2200,2500,542,32,5274
1,1,129,4,2,3,2,1,1,1100,5500,520,70,7190
2,1,38,1,1,0,1,0,0,60,950,0,13,1023
3,1,130,3,3,2,7,1,0,1200,3500,342,45,5087
4,1,75,2,2,2,6,1,0,700,1198,227,16,2141
